In [1]:
# Cell 0 — Install libs (run once)
!pip install plotly==5.15.0  # interactive charts
!pip install boto3           # only needed later if you use AWS S3
!pip install reportlab       # optional, for PDF export


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00


In [2]:
# Cell 1 — Imports and mount Drive
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import json
from datetime import datetime

# If you store the project in Google Drive, mount it:
from google.colab import drive
drive.mount('/content/drive')

# set base path to where you placed the project in your Drive
BASE = Path('/content/drive/MyDrive/student-finance-analyzer')  # change if needed
DATA_DIR = BASE / 'data'
NOTEBOOKS_DIR = BASE / 'notebooks'
SRC_DIR = BASE / 'src'
MODELS_DIR = BASE / 'models'
APP_DIR = BASE / 'app'

# Create folders if they don't exist (safe to run)
for p in [DATA_DIR, SRC_DIR, MODELS_DIR, APP_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Base path:", BASE)


Mounted at /content/drive
Base path: /content/drive/MyDrive/student-finance-analyzer


In [3]:
# Cell 2 — Load dataset (adjust filename if different)
sample_path = DATA_DIR / 'finance_dataset_modeling.csv'

# If you haven't copied the file to Drive yet, upload manually using:
# from google.colab import files; files.upload()

df = pd.read_csv(sample_path)
print("Rows:", len(df))
display(df.head())
df.info()


Rows: 212


,user_id,age,education_level,employment_status,job_title,has_loan,loan_type,savings_to_income_ratio,record_date,_year_month,usdphp_rate,cpi,deflator_to_2025-07,monthly_income_php_real_2025-07,monthly_expenses_php_real_2025-07,savings_balance_php_real_2025-07,loan_amount_php_real_2025-07,monthly_savings_php_real_2025-07
0,U00081,28,Bachelor,Self-employed,Student,1,Business,9.55,2023-08-26,2023-08,56.1599,131.5,1.056274,22516.622541,12113.694810,3.095014e+07,5.755709e+06,10402.927731
1,U00182,33,Bachelor,Employed,Student,0,NaN,6.59,2022-10-31,2022-10,58.8247,128.3,1.082619,32973.035313,13198.424002,3.127725e+07,0.000000e+00,19774.611311
2,U00228,32,Master,Employed,Student,0,NaN,6.26,2023-02-07,2023-02,54.7831,129.2,1.075077,18259.124142,7020.309781,1.646449e+07,0.000000e+00,11238.814361
3,U00455,34,Master,Self-employed,Student,1,Home,9.87,2022-04-09,2022-04,51.9760,122.5,1.133878,13369.947416,7958.519406,1.899517e+07,1.929688e+07,5411.428010
4,U00546,28,Master,Employed,Student,1,Car,9.99,2023-05-01,2023-05,55.7279,130.6,1.063553,21427.103458,7285.137452,3.081107e+07,2.144599e+06,14141.966005


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            212 non-null    object 
 1   age                                212 non-null    int64  
 2   education_level                    212 non-null    object 
 3   employment_status                  212 non-null    object 
 4   job_title                          212 non-null    object 
 5   has_loan                           212 non-null    int64  
 6   loan_type                          68 non-null     object 
 7   savings_to_income_ratio            212 non-null    float64
 8   record_date                        212 non-null    object 
 9   _year_month                        212 non-null    object 
 10  usdphp_rate                        212 non-null    float64
 11  cpi                                212 non-null    float64

In [4]:
# Cell 3 — Core calculation functions

def compute_metrics(monthly_income, monthly_expenses, savings, loan_amount, record_year=None):
    # avoid zero division
    income = float(monthly_income) if monthly_income is not None else 0.0
    expenses = float(monthly_expenses) if monthly_expenses is not None else 0.0
    savings_val = float(savings) if savings is not None else 0.0
    loan = float(loan_amount) if loan_amount is not None else 0.0

    savings_rate = (savings_val / income * 100) if income > 0 else 0.0
    expense_ratio = (expenses / income * 100) if income > 0 else 0.0
    loan_to_income_ratio = (loan / income * 100) if income > 0 else 0.0
    net_balance = income - expenses

    return {
        "monthly_income": income,
        "monthly_expenses": expenses,
        "savings": savings_val,
        "loan_amount": loan,
        "savings_rate": round(savings_rate, 2),
        "expense_ratio": round(expense_ratio, 2),
        "loan_to_income_ratio": round(loan_to_income_ratio, 2),
        "net_balance": round(net_balance, 2),
        "record_year": record_year
    }


def compute_finance_score(metrics, education_level=None):
    """
    Weighted scoring:
      - savings_rate weight 0.4
      - (100 - expense_ratio) weight 0.3
      - loan penalty: higher loan_to_income_ratio reduces score via weight 0.3
    Education-level modifiers can adjust expectations (e.g., expecting lower expenses for HS students).
    """
    sr = metrics.get("savings_rate", 0)
    er = metrics.get("expense_ratio", 100)
    lir = metrics.get("loan_to_income_ratio", 0)

    # Base components
    comp_savings = sr * 0.4
    comp_expense = (100 - er) * 0.3

    # Loan component: convert loan ratio into a score contribution; large loans penalize
    # We cap loan_penalty contribution to be between 0 and 100
    loan_component_raw = max(0, 100 - (lir * 0.5))  # lesser weight on loan ratio
    comp_loan = loan_component_raw * 0.3

    base_score = comp_savings + comp_expense + comp_loan
    # base_score is in range approx 0-100 (but may exceed slightly); normalize to 0-100
    score = max(0, min(100, round(base_score, 2)))

    # Education-level adjustment (small):
    # Example idea: expecting lower expenses for High School; for college maybe more variable
    edu = (education_level or "").strip().lower()
    if edu in ("high school", "senior high", "hs"):
        score += 3   # small bonus (lower expected expenses)
    elif edu in ("college", "undergrad", "bachelor"):
        score += 0   # neutral
    elif edu in ("masters", "postgrad", "graduate"):
        score += 2   # assume higher earning potential -> small boost
    # clamp again
    score = max(0, min(100, round(score,2)))

    # Convert to grade
    if score >= 85:
        grade = "A"
    elif score >= 70:
        grade = "B"
    elif score >= 55:
        grade = "C"
    elif score >= 40:
        grade = "D"
    else:
        grade = "E"

    return {"score": score, "grade": grade}


def analyze_finance_record(record, education_level=None):
    """
    record: dict with keys 'monthly_income','monthly_expenses','savings','loan_amount','record_year' (optional)
    returns metrics + score + simple recommendations
    """
    metrics = compute_metrics(
        record.get("monthly_income"),
        record.get("monthly_expenses"),
        record.get("savings"),
        record.get("loan_amount"),
        record.get("record_year")
    )
    scoring = compute_finance_score(metrics, education_level=education_level)
    # Recommendations based on rules
    recs = []
    if metrics["savings_rate"] >= 20:
        recs.append("Great — your savings rate is above 20%. Keep it up!")
    if metrics["expense_ratio"] > 80:
        recs.append("Your expense ratio is >80% of income. Try to cut non-essential expenses.")
    if metrics["loan_to_income_ratio"] > 100:
        recs.append("Loan amount is >100% of monthly income. Consider debt repayment planning.")
    if metrics["savings"] < (0.1 * metrics["monthly_income"]):
        recs.append("Your savings is less than 10% of income — aim to increase it gradually.")
    if metrics["net_balance"] < 0:
        recs.append("Your expenses exceed income — track expenses and seek ways to reduce them.")
    if not recs:
        recs.append("Your finances look balanced based on provided inputs.")

    out = {
        "metrics": metrics,
        "score": scoring["score"],
        "grade": scoring["grade"],
        "recommendations": recs
    }
    return out


In [5]:
# Cell 4 — Test the analysis functions with sample inputs
sample_input = {
    "monthly_income": 20000,
    "monthly_expenses": 15500,
    "savings": 4500,
    "loan_amount": 30000,
    "record_year": 2025
}
res = analyze_finance_record(sample_input, education_level="College")
import pprint
pprint.pprint(res)


{'grade': 'E',
 'metrics': {'expense_ratio': 77.5,
             'loan_amount': 30000.0,
             'loan_to_income_ratio': 150.0,
             'monthly_expenses': 15500.0,
             'monthly_income': 20000.0,
             'net_balance': 4500.0,
             'record_year': 2025,
             'savings': 4500.0,
             'savings_rate': 22.5},
 'recommendations': ['Great — your savings rate is above 20%. Keep it up!',
                     'Loan amount is >100% of monthly income. Consider debt '
                     'repayment planning.'],
 'score': 23.25}


In [6]:
# Cell 5 — Apply analysis to DataFrame (if dataset has monthly_income etc. columns)
# Change column names if your CSV uses slightly different names
INCOME_COL = 'monthly_income_php_real_2025-07'  # adjust to actual column name
EXPENSES_COL = 'monthly_expenses_php_real_2025-07'
SAVINGS_COL = 'monthly_savings_php_real_2025-07'
LOAN_COL = 'loan_amount_php_real_2025-07'
EDU_COL = 'education_level'  # adjust if different

def safe_get(row, col):
    return row[col] if col in row.index else None

# create results list
analysis_results = []
for idx, row in df.iterrows():
    record = {
        "monthly_income": safe_get(row, INCOME_COL),
        "monthly_expenses": safe_get(row, EXPENSES_COL),
        "savings": safe_get(row, SAVINGS_COL),
        "loan_amount": safe_get(row, LOAN_COL),
        "record_year": safe_get(row, 'record_date') if 'record_date' in row.index else None
    }
    edu = safe_get(row, EDU_COL)
    out = analyze_finance_record(record, education_level=edu)
    out_flat = {
        "index": idx,
        "user_id": row.get('user_id', f"user_{idx}"),
        **out["metrics"],
        "score": out["score"],
        "grade": out["grade"]
    }
    analysis_results.append(out_flat)

results_df = pd.DataFrame(analysis_results)
display(results_df.head())
results_df.describe()


,index,user_id,monthly_income,monthly_expenses,savings,loan_amount,savings_rate,expense_ratio,loan_to_income_ratio,net_balance,record_year,score,grade
0,0,U00081,22516.622541,12113.694810,10402.927731,5.755709e+06,46.20,53.80,25562.04,10402.93,2023-08-26,32.34,E
1,1,U00182,32973.035313,13198.424002,19774.611311,0.000000e+00,59.97,40.03,0.00,19774.61,2022-10-31,71.98,B
2,2,U00228,18259.124142,7020.309781,11238.814361,0.000000e+00,61.55,38.45,0.00,11238.81,2023-02-07,73.09,B
3,3,U00455,13369.947416,7958.519406,5411.428010,1.929688e+07,40.47,59.53,144330.30,5411.43,2022-04-09,28.33,E
4,4,U00546,21427.103458,7285.137452,14141.966005,2.144599e+06,66.00,34.00,10008.81,14141.97,2023-05-01,46.20,D


,index,monthly_income,monthly_expenses,savings,loan_amount,savings_rate,expense_ratio,loan_to_income_ratio,net_balance,score
count,212.000000,212.000000,212.000000,212.000000,2.120000e+02,212.000000,212.000000,212.000000,212.000000,212.000000
mean,105.500000,17408.124494,10536.367037,6871.757458,3.720031e+06,40.118868,59.881132,34000.474057,6871.757594,49.069387
std,61.343296,9074.202395,6720.457032,5151.898629,6.982148e+06,18.540173,18.540173,100190.623798,5151.898414,18.514517
min,0.000000,1800.941484,702.571804,278.382152,0.000000e+00,10.100000,30.130000,0.000000,278.380000,7.420000
25%,52.750000,10623.340894,5138.815304,3098.523912,0.000000e+00,23.315000,42.252500,0.000000,3098.522500,38.922500
50%,105.500000,16610.595400,9879.467223,5391.190376,0.000000e+00,39.585000,60.415000,0.000000,5391.190000,47.380000
75%,158.250000,24330.895893,13990.073800,9680.658686,2.826696e+06,57.747500,76.685000,26618.280000,9680.657500,63.770000
max,211.000000,38774.893197,33139.369061,25169.434524,2.600884e+07,69.870000,89.900000,912815.380000,25169.430000,81.520000


In [7]:
# Cell 6 — (Optional) export functions to src/finance_utils.py
code = r'''
# finance_utils.py
from pathlib import Path

def compute_metrics(monthly_income, monthly_expenses, savings, loan_amount, record_year=None):
    income = float(monthly_income) if monthly_income is not None else 0.0
    expenses = float(monthly_expenses) if monthly_expenses is not None else 0.0
    savings_val = float(savings) if savings is not None else 0.0
    loan = float(loan_amount) if loan_amount is not None else 0.0
    savings_rate = (savings_val / income * 100) if income > 0 else 0.0
    expense_ratio = (expenses / income * 100) if income > 0 else 0.0
    loan_to_income_ratio = (loan / income * 100) if income > 0 else 0.0
    net_balance = income - expenses
    return {
        "monthly_income": income,
        "monthly_expenses": expenses,
        "savings": savings_val,
        "loan_amount": loan,
        "savings_rate": round(savings_rate, 2),
        "expense_ratio": round(expense_ratio, 2),
        "loan_to_income_ratio": round(loan_to_income_ratio, 2),
        "net_balance": round(net_balance, 2),
        "record_year": record_year
    }

def compute_finance_score(metrics, education_level=None):
    sr = metrics.get("savings_rate", 0)
    er = metrics.get("expense_ratio", 100)
    lir = metrics.get("loan_to_income_ratio", 0)
    comp_savings = sr * 0.4
    comp_expense = (100 - er) * 0.3
    loan_component_raw = max(0, 100 - (lir * 0.5))
    comp_loan = loan_component_raw * 0.3
    base_score = comp_savings + comp_expense + comp_loan
    score = max(0, min(100, round(base_score, 2)))
    edu = (education_level or "").strip().lower()
    if edu in ("high school", "senior high", "hs"):
        score += 3
    elif edu in ("college", "undergrad", "bachelor"):
        score += 0
    elif edu in ("masters", "postgrad", "graduate"):
        score += 2
    score = max(0, min(100, round(score,2)))
    if score >= 85:
        grade = "A"
    elif score >= 70:
        grade = "B"
    elif score >= 55:
        grade = "C"
    elif score >= 40:
        grade = "D"
    else:
        grade = "E"
    return {"score": score, "grade": grade}

def analyze_finance_record(record, education_level=None):
    metrics = compute_metrics(
        record.get("monthly_income"),
        record.get("monthly_expenses"),
        record.get("savings"),
        record.get("loan_amount"),
        record.get("record_year")
    )
    scoring = compute_finance_score(metrics, education_level=education_level)
    recs = []
    if metrics["savings_rate"] >= 20:
        recs.append("Great — your savings rate is above 20%. Keep it up!")
    if metrics["expense_ratio"] > 80:
        recs.append("Your expense ratio is >80% of income. Try to cut non-essential expenses.")
    if metrics["loan_to_income_ratio"] > 100:
        recs.append("Loan amount is >100% of monthly income. Consider debt repayment planning.")
    if metrics["savings"] < (0.1 * metrics["monthly_income"]):
        recs.append("Your savings is less than 10% of income — aim to increase it gradually.")
    if metrics["net_balance"] < 0:
        recs.append("Your expenses exceed income — track expenses and seek ways to reduce them.")
    if not recs:
        recs.append("Your finances look balanced based on provided inputs.")
    out = {
        "metrics": metrics,
        "score": scoring["score"],
        "grade": scoring["grade"],
        "recommendations": recs
    }
    return out
'''

# write file to src
file_path = SRC_DIR / 'finance_utils.py'
file_path.write_text(code)
print("Wrote:", file_path)


Wrote: /content/drive/MyDrive/student-finance-analyzer/src/finance_utils.py


In [8]:
# Cell 7 — Simulate user input dictionary and analyze
user_input = {
    "monthly_income": float(input("Enter monthly income (₱): ") or 20000),
    "monthly_expenses": float(input("Enter monthly expenses (₱): ") or 15500),
    "savings": float(input("Enter current savings (₱): ") or 4500),
    "loan_amount": float(input("Enter loan amount (₱): ") or 30000),
    "record_year": int(input("Record Year (e.g., 2025): ") or 2025)
}
edu = input("Education Level (College / High School / Masters): ") or "College"

analysis = analyze_finance_record(user_input, education_level=edu)
from pprint import pprint
pprint(analysis)


Enter monthly income (₱): 40000
Enter monthly expenses (₱): 20000
Enter current savings (₱): 50000
Enter loan amount (₱): 0
Record Year (e.g., 2025): 2025
Education Level (College / High School / Masters): College
{'grade': 'A',
 'metrics': {'expense_ratio': 50.0,
             'loan_amount': 0.0,
             'loan_to_income_ratio': 0.0,
             'monthly_expenses': 20000.0,
             'monthly_income': 40000.0,
             'net_balance': 20000.0,
             'record_year': 2025,
             'savings': 50000.0,
             'savings_rate': 125.0},
 'recommendations': ['Great — your savings rate is above 20%. Keep it up!'],
 'score': 95.0}
